In [4]:
import numpy as np

def simplex(c, A, b):
    nrows, ncols = A.shape
    tableau = np.zeros((nrows + 1, ncols + nrows + 1))
    tableau[:nrows, :ncols] = A
    tableau[:nrows, ncols:ncols + nrows] = np.eye(nrows)
    tableau[:nrows, -1] = b
    tableau[-1, :ncols] = -np.array(c)

    basis = list(range(ncols, ncols + nrows))
    while any(tableau[-1, :-1] < 0):
        col = np.argmin(tableau[-1, :-1])
        ratios = [tableau[i, -1] / tableau[i, col] if tableau[i, col] > 0 else float('inf') for i in range(nrows)]
        row = np.argmin(ratios)
        pivot = tableau[row, col]
        tableau[row, :] /= pivot
        for i in range(nrows + 1):
            if i != row:
                tableau[i, :] -= tableau[i, col] * tableau[row, :]
        basis[row] = col
                
    solution = np.zeros(ncols)  
    for i, b_col in enumerate(basis):
        if b_col < ncols:
            solution[b_col] = tableau[i, -1]
    return tableau[-1, -1], solution

c = [8, 7, 6]
A = np.array([
 [2, 3, 4],
 [1, 4, 5],
 [3, 4, 2]
])
b = [700, 800, 600]

optimal_value, solution = simplex(c, A, b)
print("Максимальная прибыль:", optimal_value)
print("Распределение производства:", solution)

Максимальная прибыль: 1675.0
Распределение производства: [125.    0.  112.5]


In [5]:
D = np.array([
 [4, 0, 2],
 [5, 1, 1],
 [2, 0, 3]
])
c = np.array([6, 0, 8])
D_inv = np.linalg.inv(D)

y = c.dot(D_inv)
gmin = b*y
gmin.sum()

1675.0

In [6]:
import numpy as np

y = np.linalg.solve(D.T, c)

print(y)

[ 0.25 -0.    2.5 ]


In [36]:
import numpy as np

indices = np.array([112.5, 112.5, 125])

b = np.array([700, 800, 600])

# Ресурс 1
positive_mask = D_inv[:, 2] > 0
negative_mask = D_inv[:, 2] < 0
delta_b1_min = max(abs((indices[positive_mask] / D_inv[positive_mask, 2])))
delta_b1_max = max(abs((indices[negative_mask] / D_inv[negative_mask, 2])))
b1_interval = [b[0] - delta_b1_min, b[0] + delta_b1_max]

# Ресурс 2
positive_mask = D_inv[:, 1] > 0
if np.any(positive_mask): 
    delta_b2_min = -1 * max((indices[positive_mask] / D_inv[positive_mask, 1]))
else:
    delta_b2_min = float('inf') 
b2_interval = [b[1] + delta_b2_min, np.inf]

# Ресурс 3
positive_mask = D_inv[:, 0] > 0
negative_mask = D_inv[:, 0] < 0
delta_b3_min = max(abs((indices[positive_mask] / D_inv[positive_mask, 0])))
delta_b3_max = max(abs((indices[negative_mask] / D_inv[negative_mask, 0])))

b3_interval = [b[2] - delta_b3_min, b[2] + delta_b3_max]

print(f"Интервал устойчивости для ресурса 1: {b1_interval}")
print(f"Интервал устойчивости для ресурса 2: {b2_interval}")
print(f"Интервал устойчивости для ресурса 3: {b3_interval}")
g1= y[0] * delta_b1_max
g2 = y[2] * delta_b3_max
print(delta_b3_max)
gmax = optimal_value+g1+g2

print(g1, g2, gmax)

Интервал устойчивости для ресурса 1: [450.0, 1150.0]
Интервал устойчивости для ресурса 2: [687.5, inf]
Интервал устойчивости для ресурса 3: [300.0, 1100.0]
500.0
112.5 1250.0 3037.5


In [15]:
class MealyError(Exception):
    def __init__(self, method_name):
        super().__init__(f"Method '{method_name}' not allowed")

class MealyMachine:
    def __init__(self):
        self.state = 'A'

    def step(self):
        if self.state == 'A':
            self.state = 'B'
            return 0
        elif self.state == 'B':
            self.state = 'C'
            return 2
        elif self.state == 'C':
            self.state = 'D'
            return 5
        else:
            raise MealyError('step')

    def drag(self):
        if self.state == 'B':
            self.state = 'A'
            return 3
        elif self.state == 'E':
            self.state = 'F'
            return 7
        else:
            raise MealyError('drag')

    def color(self):
        if self.state == 'A':
            self.state = 'E'
            return 1
        elif self.state == 'D':
            self.state = 'E'
            return 6
        elif self.state == 'F':
            return 8
        else:
            raise MealyError('color')

def main():
    return MealyMachine()

def test():
    test_cases = [
        ("step,drag,step,step,color,drag,color,drag,color,color,color", [0, 3, 0, 5, 6, "MealyError", 8, "MealyError", 8, 8, 8])
    ]

    for actions, expected in test_cases:
        o = main()
        results = []
        actions = actions.split(',')
        for action, exp in zip(actions, expected):
            try:
                result = eval(f'o.{action}()')
                assert result == exp, f"Expected {exp}, got {result}"
                results.append(result)
            except MealyError as me:
                if exp == "MealyError":
                    results.append("MealyError")
                else:
                    raise AssertionError(f"Unexpected MealyError during {action}")
        print(f"Test case: [{', '.join(actions)}] passed with results: {results}")

# Run the test function
test()


AssertionError: Expected 5, got 2